# Cleaning Data
## Wellcome Journal Dataset
### Sam Fradkin

In [57]:
import pandas as pd
import numpy as np
import scipy.stats

In [58]:
# It appears the CSV is using Latin_1 encoding
data = pd.read_csv('Wellcome_Data.csv', encoding='latin_1')
 
data.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [59]:
# Rename the Cost column
data.rename(inplace=True, columns={'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'})

In [60]:
# Remove all whitespace at the beginning and ends of cells
for column in data.columns:
    data[column] = data[column].str.strip()

data['PMID/PMCID'] = data['PMID/PMCID'].str.replace('\s*', "")

In [61]:
# Remove all unnecessary parts of the COST values to be able to convert them to integers
data['Cost'] = data['Cost'].str.replace('[£$]*', "")
data['Cost'] = data['Cost'].str.replace('[$£]?[.][0-9]{0,2}[$£]*', "")

In [69]:
# Change case of all journal names so they can aggregate properly
data['Journal title'] = data['Journal title'].str.title()

In [70]:
# Find out how many rows have NULL values
print(f"The number of rows that contains null values in PMID column is: {data.isnull().values.ravel().sum()}")

The number of rows that contains null values in PMID column is: 0


In [71]:
# Get rid of those rows
data.dropna(inplace=True)

In [72]:
# Convert COST values to integers
data.Cost = pd.to_numeric(data.Cost, errors='coerce').fillna(0).astype(np.int64)

In [73]:
# Remove all extreme outliers from the data through Winsorization
scipy.stats.mstats.winsorize(data.Cost, limits=0.03, inplace=True)

masked_array(data=[2381,  642,  669, ..., 1834, 2374, 2034],
             mask=False,
       fill_value=999999)

In [78]:
# Create a list of the 5 most common journals
FMCJournals = list(pd.value_counts(data['Journal title']).head(5).index)

In [82]:
# Run a for loop through the list of five most common journals and pull their
# Means, Medians, and Standard Deviations
for cost, journal in enumerate(FMCJournals):
    stats = data.loc[data['Journal title'] == FMCJournals[cost], 'Cost']
    print(f"{FMCJournals[cost].title()} - Mean: £{stats.mean():.0f}")
    print(f"{FMCJournals[cost].title()} - Median: £{stats.median():.0f}")
    print(f"{FMCJournals[cost].title()} - St Dev: £{stats.std(ddof=1):.0f}\n")

Plos One - Mean: £1079
Plos One - Median: £900
Plos One - St Dev: £656

Journal Of Biological Chemistry - Mean: £1484
Journal Of Biological Chemistry - Median: £1328
Journal Of Biological Chemistry - St Dev: £538

Neuroimage - Mean: £2230
Neuroimage - Median: £2334
Neuroimage - St Dev: £258

Nucleic Acids Research - Mean: £1161
Nucleic Acids Research - Median: £852
Nucleic Acids Research - St Dev: £448

Plos Pathogens - Mean: £1781
Plos Pathogens - Median: £1600
Plos Pathogens - St Dev: £723

